In [2]:
from contextlib import contextmanager
from dataclasses import dataclass
from typing import Callable, List, Optional, Tuple, Union, Literal

In [3]:
def name_generator(f):
    '''Just a little name generator so we can tell our things apart!'''
    i = 0
    while True:
        yield f(i)
        i += 1
names = name_generator(lambda n: f"thing-{n}")

`MhThingy` is just a little thing to get from our context manager, that
will let us see how it works and how it interacts with exception handling.

The `compute()` method takes a keyword argument `fail=`, which if true,
causes it to raise an exception.

It also raises an exception if it is not ready. It is only ready while within
the context manager's scope.

In [4]:
@dataclass
class MyThingy:
    name: str = next(names)
    exception: Optional[Exception] = None
    state: Literal['new', 'ready', 'done'] = 'new'
    def compute(self, /, fail=False) -> int:
        print(f'MT: compute {self}')
        if fail:
            raise ValueError(f"> {self} failed")
        if self.state == 'ready':
            return 42
        else:
            raise Exception("Not ready: {self}")

We can create an instance of MyThingy, but it won't be marked as ready, and we
won't know what to do to make it ready, nor unready.

In [5]:
MyThingy()

MyThingy(name='thing-0', exception=None, state='new')

Here is our context manager. There are many was to implemnt context managers, but using
a generator and the @contextmanager decorator is one of the simplest.

I'll cover an even simpler one at the end. But first, let's look at a generator-based context
manager, how generators work.

In [6]:
@contextmanager
def my_context_manager():
    print("Enter")
    try:
        thing = MyThingy()
        thing.state = 'ready'
        # We don't care about the return value for yield here
        print("CM: yielding {thing}")
        result = yield thing
        print("CM: received {result}.")
    except Exception as ex:
        # ContextManagers don't usually need to catch exceptions,
        # but this helps us see what is happening.
        print(f"CM: got an exception: {repr(ex)}.")
        thing.exception = ex
        raise ex
    finally:
        print(f'CM: Done with {thing}.')
        thing.state = 'done'
        print("CM: Exit")

Let's use in a `with` statement.

In [7]:
def do_thing(*, fail: bool = False) -> MyThingy:
    with my_context_manager() as athing:
        value = athing.compute(fail=fail)
        print(f"DT: In context: {athing} => {value}")
    # Bad idea: using athing outside the with statement
    print(f"DT: Out of context: {athing}")
    # Worse idea: returning it!
    return athing

Let's first call `do_thing` without an exception.

In [8]:
a_thing = do_thing()
print(f"TL: Out of context: {a_thing}")

Enter
CM: yielding {thing}
MT: compute MyThingy(name='thing-0', exception=None, state='ready')
DT: In context: MyThingy(name='thing-0', exception=None, state='ready') => 42
CM: received {result}.
CM: Done with MyThingy(name='thing-0', exception=None, state='ready').
CM: Exit
DT: Out of context: MyThingy(name='thing-0', exception=None, state='done')
TL: Out of context: MyThingy(name='thing-0', exception=None, state='done')


Let's run it again, but with `fail=True` so we can look at the flow
when there's an exception:

In [9]:
do_thing(fail=True)

Enter
CM: yielding {thing}
MT: compute MyThingy(name='thing-0', exception=None, state='ready')
CM: got an exception: ValueError("> MyThingy(name='thing-0', exception=None, state='ready') failed").
CM: Done with MyThingy(name='thing-0', exception=ValueError("> MyThingy(name='thing-0', exception=None, state='ready') failed"), state='ready').
CM: Exit


ValueError: > MyThingy(name='thing-0', exception=None, state='ready') failed

I labelled the print statements to make it easier to interpret the output:

* `CM`: The context manager generatof function.
* `MT`: The `MyThing.compute()` method.
* `DT`: The `do_thing()` function
* `TL`: Top level

From our discussion yesterday, you should understand basically how the above works. So let's
look at how the generator is working as a context manager, with the help of the `@contextmanager` decorator.

Let's copy it without the `@contextmanager` decorator.

In [ ]:
def almost_context_manager():
    print("Enter")
    try:
        thing = MyThingy()
        thing.state = 'ready'
        # We don't care about the return value for yield here
        print("CM: yielding {thing}")
        result = yield thing
        print("CM: received {result}.")
    except Exception as ex:
        # ContextManagers don't usually need to catch exceptions,
        # but this helps us see what is happening.
        print(f"CM: got an exception: {repr(ex)}.")
        thing.exception = ex
        raise ex
    finally:
        print(f'CM: Done with {thing}.')
        thing.state = 'done'
        print("CM: Exit")

In [ ]:
def demo(path: Literal['next', 'send', 'throw', 'end']):
    # Before decoration, my_context_manager() returns a generator.
    cm = almost_context_manager()
    # From the generator, we can get an iterator
    cm_it = iter(cm)
    try:
        match path:
            case 'next':
                ## next() is used to get the next element in
                ## iterating, etc.
                ## It is also used to get the object for the 'as <var>' part
                ## of the `with` statement.
                print(f"Next: {next(cm_it)}")
            case 'send':
                ## You can't send before the generator has yielded its first
                ## value. (Onl value, if we're makinog a context manager)
                print(f"Next: {next(cm_it)}")
                ## send and next are two sides of the same coin. Send
                ## gives a return value for `yield`, `next` gives the
                ## hielded value.
                print(f"Send returned: {cm.send('OK')}")
            case 'throw':
                # @contextmanager uses .throw(ex) to pass the exception
                # to the generator function.
                # After doing a `yield`, a generator is pausedd, only waking up
                # when send or next are done.
                # an exception to throw inside the generator
                # This will then continue on up the stack to be handled
                # outside (or not).
                ex = ArithmeticError("the answer != 42")
                cm.throw(ex)
            case 'end':
                # Iterators throw StopIteration when next() is called
                # and they don't have any more values.
                print(f"Next #1: {next(cm_it)}")
                print(f"Next #2: {next(cm_it)}")
            case _:
                print(f"Invalid arg: {path}. It should be one of 'next', 'send','throw' or 'end'")
    except StopIteration:
        print("Done iterating")
    except Exception as ex:
        print(f"We got back the error we sent: {repr(ex)}")


In [ ]:
demo('next')

In [ ]:
demo('send')

In [ ]:
demo('throw')

So how do we turn a generator into a context manager?

A context manager just implements`__enter__` and `__exit__` methods.

So let's make a decorator!

In [ ]:
# I'll skip the type annotations to keep it simple.
class OurCtxMgr:
    def __init__(self, gen):
        print(f"OC: Creating real context manager to wrap {gen}")
        self.gen = gen
    def __enter__(self):
        # This is what the next() function does.
        return self.gen.__next__()
    def __exit__(self, type, value, traceback):
        if type is None:
            # Let the generator continue.
            # I use send() to illustrate, but the real
            # code uses __next__(), bcause there's nothing
            # useful to send
            try:
                self.gen.send("norhing useful")
            except StopIteration:
                # Either way, we have to handle StopIteration.
                print(f"OC: Generator completed")
        else:
            print(f"C: Passing exception {value} on to generator.")
            try:
                self.gen.throw(type, value, traceback)
            except Exception as ex:
                print(f"OC: Generator's uncaught exception: {ex}")
                raise ex
def our_ctx_mgr(f):
    def ctx_mgr(*args, **kwargs):
        return OurCtxMgr(f(*args, **kwargs))
    # Return our inner function to use in place of the supplied function.
    return ctx_mgr

So let's use it to make a real context manager!

In [ ]:
@our_ctx_mgr
def real_context_manager():
    print("Enter")
    try:
        thing = MyThingy()
        thing.state = 'ready'
        # We don't care about the return value for yield here
        print(f"CM: yielding {thing}")
        result = yield thing
        print(f"CM: received {result}.")
    except Exception as ex:
        # ContextManagers don't usually need to catch exceptions,
        # but this helps us see what is happening.Z
        print(f"CM: got an exception: {repr(ex)}.")
        thing.exception = ex
        raise ex
    finally:
        print(f'CM: Done with {thing}.')
        thing.state = 'done'
        print("CM: Exit")
        

In [ ]:
def test_our_thing(fail:bool = False):
    with real_context_manager() as our_thing:
        print(f"TO: got {our_thing}")
        val = our_thing.compute(fail=fail)
        print(f"TO: Computed {val}")
        return our_thing # Still a bad idea!

In [ ]:
test_our_thing(fail=False)

In [ ]:
test_our_thing(fail=True)

If you carefully examine that stack trace, you can see tha the generator was waiting at the `yield` statement
for the outside to ask for the `next` value (or `send` it one, `next` is really just `return send(None)`

## An even simpler context manager

I promised you earlier an even simpler context manager.

First, let's make one little addition to `MyThing`

In [ ]:
class SimpleThing(MyThingy):
    def close(self):
        print("ST: closing {self}")
        self.state = 'done'

def make_ready(thing: SimpleThing):
    match thing.state:
        case 'new':
            print(f"MR: Making it ready.")
            thing.state = 'ready'
            return thing
        case 'ready':
            return thing
        case 'done':

            raise Exception(f"{thing} is already done")

Now let's see how to make a context manager out of that on the fly!

In [ ]:
:from contextlib import closing
with closing(make_ready(SimpleThing())) as st:
    print(f"TL: {st} => {st.compute()}")

### Bonus Round

In [16]:
@contextmanager
def until_end():
    try:
        yield
    except StopIteration:
        print("Iteration done!")

In [17]:
it = iter((1, 2, 3))
with until_end():
    while True:
        print(next(it))

1
2
3
Iteration done!


Our context manager handled the exception for us!

## Conclusion

Generators are more flexible than using `closing()`

    * They handle initialization. Without them we had to introduce s `make_ready` function'
    * They can handle exceptions more flexibly.
    * They are consice and easy to write.
    * They do not depend on anything about the object they expose.
        - They only need to be able to create or locate it and yield it.